# การสร้างข้อมูล Preference Dataset

## สารบัญ
1. [บทนำ](#1-บทนำ)
2. [วิธีการสร้างข้อมูล Preference](#2-วิธีการสร้างข้อมูล-preference)
3. [ติดตั้ง Ollama](#3-ติดตั้ง-ollama)
4. [สร้างข้อมูลด้วย LLM](#4-สร้างข้อมูลด้วย-llm)
5. [ตัวอย่างข้อมูลภาษาไทย](#5-ตัวอย่างข้อมูลภาษาไทย)

---
## 1. บทนำ

**Preference Finetuning** เป็นกระบวนการจัดแนว LLM ที่ผ่านการ Instruction Finetuning แล้ว ให้ตรงกับความชอบของมนุษย์มากขึ้น

### ข้อมูลที่ต้องการ

สำหรับ DPO เราต้องการข้อมูลในรูปแบบ:

```json
{
    "instruction": "คำสั่งหรือคำถาม",
    "input": "ข้อมูลเพิ่มเติม (optional)",
    "chosen": "คำตอบที่ดี (ที่เราต้องการ)",
    "rejected": "คำตอบที่ไม่ดี (ที่เราไม่ต้องการ)"
}
```

### ตัวอย่าง

```json
{
    "instruction": "อธิบายเกี่ยวกับประเทศไทย",
    "input": "",
    "chosen": "ประเทศไทยตั้งอยู่ในเอเชียตะวันออกเฉียงใต้ มีเมืองหลวงคือกรุงเทพฯ เป็นประเทศที่มีวัฒนธรรมอันเก่าแก่และเป็นที่รู้จักในเรื่องอาหารอร่อย",
    "rejected": "ไทยก็อยู่แถวๆ เอเชีย กรุงเทพเป็นเมืองหลวง ก็แค่นั้น"
}
```

---
## 2. วิธีการสร้างข้อมูล Preference

มี 3 วิธีหลักในการสร้างข้อมูล:

### วิธีที่ 1: ให้มนุษย์จัดอันดับ
- ใช้ LLM สร้างคำตอบหลายๆ แบบ
- ให้มนุษย์จัดอันดับว่าแบบไหนดีกว่า
- **ข้อดี**: ได้ข้อมูลคุณภาพสูง
- **ข้อเสีย**: ใช้เวลาและค่าใช้จ่ายสูง

### วิธีที่ 2: ใช้ LLM จัดอันดับ
- ใช้ LLM สร้างคำตอบหลายๆ แบบ
- ใช้ LLM อีกตัว (หรือตัวเดียวกัน) จัดอันดับ
- **ข้อดี**: เร็วและราคาถูกกว่า
- **ข้อเสีย**: อาจมี bias

### วิธีที่ 3: ใช้ LLM สร้างคู่ตรงๆ ⬅️ (ใช้ใน notebook นี้)
- กำหนดเกณฑ์ให้ LLM (เช่น สุภาพ vs ไม่สุภาพ)
- ให้ LLM สร้างทั้ง chosen และ rejected
- **ข้อดี**: เร็วที่สุด
- **ข้อเสีย**: ต้องระวังคุณภาพ

---
## 3. ติดตั้ง Ollama

**Ollama** เป็นเครื่องมือสำหรับรัน LLM บนเครื่องของเรา

### ขั้นตอนการติดตั้ง:

1. ไปที่ https://ollama.com
2. ดาวน์โหลดและติดตั้งตามระบบปฏิบัติการของคุณ
3. เปิด Terminal และรัน:

```bash
# ดาวน์โหลดโมเดล Llama 3.1 (8B)
ollama pull llama3.1

# หรือถ้ามี RAM เยอะ ใช้ 70B
ollama pull llama3.1:70b
```

4. ทดสอบ:

```bash
ollama run llama3.1
>>> สวัสดีครับ
```

In [ ]:
# Import libraries
import json
import requests
import random
from tqdm import tqdm

In [ ]:
def query_ollama(prompt, model="llama3.1", url="http://localhost:11434/api/chat"):
    """
    ส่ง prompt ไปหา Ollama และรับคำตอบกลับมา
    
    Parameters:
    -----------
    prompt: str
        ข้อความที่จะส่งไป
    model: str
        ชื่อโมเดล (default: llama3.1)
    url: str
        URL ของ Ollama API
    
    Returns:
    --------
    str: คำตอบจากโมเดล
    """
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "options": {
            "seed": 123,
            "temperature": 0.7,
        }
    }
    
    try:
        with requests.post(url, json=data, stream=True, timeout=60) as r:
            r.raise_for_status()
            response_data = ""
            for line in r.iter_lines(decode_unicode=True):
                if not line:
                    continue
                response_json = json.loads(line)
                if "message" in response_json:
                    response_data += response_json["message"]["content"]
        return response_data
    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
# ทดสอบการเชื่อมต่อ Ollama
# หมายเหตุ: ต้องรัน 'ollama serve' ก่อนใน terminal อื่น

test_response = query_ollama("สวัสดีครับ แนะนำตัวหน่อย")
if test_response:
    print("✅ เชื่อมต่อ Ollama สำเร็จ!")
    print(f"\nคำตอบ:\n{test_response}")
else:
    print("❌ ไม่สามารถเชื่อมต่อ Ollama ได้")
    print("กรุณาตรวจสอบว่า:")
    print("1. ติดตั้ง Ollama แล้ว")
    print("2. รัน 'ollama serve' ใน terminal")
    print("3. ดาวน์โหลดโมเดลด้วย 'ollama pull llama3.1'")

---
## 4. สร้างข้อมูลด้วย LLM

เราจะใช้ LLM สร้างคู่ของ chosen/rejected responses

### เกณฑ์ที่ใช้:
- **Chosen**: คำตอบที่สุภาพ ละเอียด เป็นประโยชน์
- **Rejected**: คำตอบที่สั้นเกินไป ไม่สุภาพ หรือไม่เป็นประโยชน์

In [ ]:
def format_input(entry):
    """
    จัดรูปแบบ prompt
    """
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    
    input_text = f"\n\n### Input:\n{entry['input']}" if entry.get("input") else ""
    
    return instruction_text + input_text

In [ ]:
def generate_preference_pair(entry, model="llama3.1"):
    """
    สร้างคู่ chosen/rejected สำหรับ entry หนึ่ง
    
    Parameters:
    -----------
    entry: dict
        ข้อมูลที่มี instruction และ output
    model: str
        โมเดลที่จะใช้
    
    Returns:
    --------
    dict: entry ที่มี chosen และ rejected เพิ่มเติม
    """
    formatted_input = format_input(entry)
    original_output = entry.get("output", "")
    
    # สุ่มว่าจะสร้างแบบสุภาพหรือไม่สุภาพ
    politeness = random.choice(["polite", "impolite"])
    
    prompt = f"""Given the following instruction and original response, rewrite the response to be more {politeness}.

Instruction: {entry['instruction']}
Original Response: {original_output}

Rules:
- If 'polite': Make it more helpful, detailed, and professional
- If 'impolite': Make it short, dismissive, or slightly rude (but still correct)
- Keep the core information the same
- Only return the rewritten response, nothing else
- Respond in the same language as the original

Rewritten response:"""
    
    rewritten = query_ollama(prompt, model=model)
    
    if rewritten:
        if politeness == "polite":
            entry["chosen"] = rewritten.strip()
            entry["rejected"] = original_output
        else:
            entry["chosen"] = original_output
            entry["rejected"] = rewritten.strip()
    else:
        # Fallback: ใช้ original เป็น chosen
        entry["chosen"] = original_output
        entry["rejected"] = original_output[:len(original_output)//2]  # ตัดครึ่ง
    
    return entry

In [ ]:
# ตัวอย่างการใช้งาน
sample_entry = {
    "instruction": "อธิบายว่า Python คืออะไร",
    "input": "",
    "output": "Python เป็นภาษาโปรแกรมมิ่ง"
}

print("ก่อนสร้าง Preference:")
print(json.dumps(sample_entry, indent=2, ensure_ascii=False))

# สร้าง preference pair (ต้องมี Ollama รันอยู่)
# result = generate_preference_pair(sample_entry)
# print("\nหลังสร้าง Preference:")
# print(json.dumps(result, indent=2, ensure_ascii=False))

In [ ]:
def generate_preference_dataset(data, model="llama3.1", max_entries=None):
    """
    สร้าง preference dataset จากข้อมูล instruction
    
    Parameters:
    -----------
    data: list
        รายการของ entries
    model: str
        โมเดลที่จะใช้
    max_entries: int, optional
        จำนวน entries สูงสุด (ถ้าไม่ระบุจะใช้ทั้งหมด)
    
    Returns:
    --------
    list: preference dataset
    """
    if max_entries:
        data = data[:max_entries]
    
    preference_data = []
    
    for entry in tqdm(data, desc="กำลังสร้างข้อมูล"):
        result = generate_preference_pair(entry.copy(), model=model)
        preference_data.append(result)
    
    return preference_data

---
## 5. ตัวอย่างข้อมูลภาษาไทย

นี่คือตัวอย่างข้อมูล Preference ภาษาไทยที่สร้างไว้แล้ว

In [ ]:
# โหลดข้อมูลตัวอย่าง
with open("dpo_thai_dataset.json", "r", encoding="utf-8") as f:
    thai_dpo_data = json.load(f)

print(f"จำนวนข้อมูล: {len(thai_dpo_data)} รายการ")

In [ ]:
# แสดงตัวอย่าง
for i in range(min(3, len(thai_dpo_data))):
    entry = thai_dpo_data[i]
    print("="*60)
    print(f"ตัวอย่างที่ {i+1}")
    print("="*60)
    print(f"\n📝 Instruction: {entry['instruction']}")
    if entry.get('input'):
        print(f"📥 Input: {entry['input']}")
    print(f"\n✅ Chosen (คำตอบที่ดี):")
    print(f"{entry['chosen']}")
    print(f"\n❌ Rejected (คำตอบที่ไม่ดี):")
    print(f"{entry['rejected']}")
    print()

### บันทึกข้อมูล

In [ ]:
# บันทึกข้อมูลเป็น JSON
def save_preference_data(data, filename):
    """
    บันทึกข้อมูลเป็นไฟล์ JSON
    """
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✅ บันทึกข้อมูลไปที่: {filename}")

# save_preference_data(preference_data, "my_preference_data.json")

---
## สรุป

### สิ่งที่เราได้เรียนรู้:

1. **รูปแบบข้อมูล Preference**: ต้องมี instruction, chosen, rejected

2. **วิธีสร้างข้อมูล**: 
   - มนุษย์จัดอันดับ
   - LLM จัดอันดับ
   - LLM สร้างคู่โดยตรง

3. **ใช้ Ollama**: รัน LLM บนเครื่องเพื่อสร้างข้อมูล

4. **เกณฑ์ความชอบ**: สามารถกำหนดได้หลายแบบ เช่น:
   - สุภาพ vs ไม่สุภาพ
   - ละเอียด vs สั้น
   - เป็นประโยชน์ vs ไม่เป็นประโยชน์

---

**ขั้นตอนถัดไป**: ไปยัง `01_dpo_from_scratch_thai.ipynb` เพื่อฝึกโมเดลด้วย DPO

---

**อ้างอิง:**

- Raschka, Sebastian. *Build A Large Language Model (From Scratch)*. Manning, 2024. ISBN: 978-1633437166.
- https://github.com/rasbt/LLMs-from-scratch
- https://ollama.com

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<b>การสร้างข้อมูล Preference สำหรับ DPO</b><br>
แปลและปรับปรุงจากหนังสือ <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> โดย <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Repository ต้นฉบับ: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

---

**อ้างอิง:**
> Raschka, Sebastian. *Build A Large Language Model (From Scratch)*. Manning, 2024. ISBN: 978-1633437166.

---